In [ ]:
using Pkg
envpath = expanduser("~/envs/d11/")
Pkg.activate(envpath)

# libs
using Images
using DataFrames
using CSV
using Random
using MLUtils: splitobs, randobs
using FLoops

# private libs
using CocoTools
using PreprocessingImages; const pi=PreprocessingImages

In [ ]:
# input folders
datasetpath  = "../coco/"
trainsetpath = datasetpath * "train2017/"
testsetpath  = datasetpath * "val2017/"

# output folders
workpath = pwd() * "/"
workpath = replace(workpath, homedir() => "~")

In [ ]:
# create full data frames
trainImages = filter(x -> occursin(r"jpg", x), readdir(expanduser(trainsetpath)))
trainMasks  = filter(x -> occursin(r"png", x), readdir(expanduser(trainsetpath)))
testImages  = filter(x -> occursin(r"jpg", x), readdir(expanduser(testsetpath)))
testMasks   = filter(x -> occursin(r"png", x), readdir(expanduser(testsetpath)))

fpfnTrainImages = trainsetpath .* trainImages
fpfnTrainMasks  = trainsetpath .* trainMasks
fpfnTestImages  = testsetpath .* testImages
fpfnTestMasks   = testsetpath .* testMasks

# create data frames
dftrain = DataFrame(X=fpfnTrainImages, y=fpfnTrainMasks)
dftest  = DataFrame(X=fpfnTestImages, y=fpfnTestMasks)

size(dftrain), size(dftest)

In [ ]:
first(dftrain, 5)

In [ ]:
first(dftest, 5)

In [ ]:
# show image and mask
fpfn = expanduser(dftrain.X[1])
img = load(fpfn)
img = imresize(img, ratio=1/4)

fpfn = expanduser(dftrain.y[1])
mask = load(fpfn)
mask = imresize(mask, ratio=1/4)

mosaicview(img, mask; nrow=1)

In [ ]:
# save data frames
CSV.write(expanduser(workpath) * "dftrain.csv", dftrain)
CSV.write(expanduser(workpath) * "dftest.csv", dftest)